# Depletion, Activation, and Decay

Materials placed in a neutron flux will undergo transmutation, depletion, and if radioactive, radioactive decay. Changes to composition will therefore also change $\Sigma$, which influences the neutron flux and reactivity. It is crucial that we solve the depletion equations accurately, in order to (i) compensate for fission product poisons with sufficient excess reactivity, (ii) optimize the fuel costs, and (iii) develop operational strategies to mitigate effects of xenon.

The governing equations for nuclide concentration are referred to as the ``Bateman equations,'' and they represent for each nuclide species, rates of production and loss.


$$ \frac{dN_i(t)}{dt}=\underbrace{\sum_{j=1,\ \text{all nuclides}}\left\lbrack f_{j\rightarrow i}\int_0^\infty dE\sigma_j(E,t)\phi(E,t)N_j(t)\right\rbrack}_{1}+\underbrace{\sum_{j=1,\ \text{all nuclides}}\left\lbrack \lambda_{j\rightarrow i}N_j(t)\right\rbrack}_{2}\\-\underbrace{\int_0^\infty dE\sigma_i(E,t)\phi(E,t)N_i(t)}_{3}-\underbrace{\sum_{j=1,\ \text{all nuclides}}\lambda_{i\rightarrow j}N_i(1)}_{4} $$



1. *Transmutation Production*: production of nuclide $i$ from reactions involving nuclide $j$. Only a fraction, $f_{j\rightarrow i}$, produce nuclide $i$.
2. *Decay Production*: production of nuclide $i$ from decay of all other nuclides $j$.
3. *Transmutation Loss*: loss of nuclide $i$ from nuclear interactions.
4. *Decay Loss*: decay of nuclide $i$

For this demonstration, we will use a simple LWR fuel pin.

In [ ]:
import math
import matplotlib.pyplot as plt
import openmc.deplete

In [ ]:
model = openmc.examples.pwr_pin_cell()
model.geometry.root_universe.plot(width=(1.26, 1.26))

model.settings.particles = 1000
model.settings.inactive = 10
model.settings.batches = 50

The depletion interface relies on OpenMC to perform the transport simulation and obtain reaction rates and other important information. 

We must first add one bit of information: the volume of any depletable regions. In order to translate the reaction rates obtained by OpenMC to meaningful units for depletion, we have to normalize them to a correct power. This requires us to know, or be able to calculate, how much depletable material is in our problem. Correctly setting the volumes is a critical step, and can lead to incorrect answers, as the fuel is over- or under-depleted due to poor normalization.

For our problem, we can assign the "volume" to be the cross-sectional area of our fuel. This is identical to modeling our fuel pin inside a box with height of 1 cm.

## The `Chain` class

The OpenMC depletion interface can be accessed from the `openmc.deplete` module, and has a variety of classes that will help us. In order to run the depletion calculation we need the following information:

1. Decay modes and half-lives, fission product yields (FPYs), and reaction data
2. Operational power, power density, or source rate (for fixed source calculations)
3. Desired depletion schedule
4. Desired time integration scheme

The additional nuclear data (decay and FPY) are stored in a "depletion chain" XML file that is generated from a combination of the `IncidentNeutron`, `Decay`, and `FissionProductYields` classes. The `openmc.deplete.Chain` class aggregates the necessary data. While the `Chain` class has a `from_endf` method that can be used, you may prefer to download pre-generated XML-representations instead at https://openmc.org/depletion-chains/.

For this session, we will be using a much smaller depletion chain that contains very few nuclides that results in shorter simulations.

Generally, we won't need to instantiate the `Chain` class directly since it is handled by other classes under the hood.

## The Depletion `CoupledOperator`

The primary entry point for depletion is the `openmc.deplete.CoupledOperator` class. It relies on the `openmc.deplete.Chain` and helper classes to run `openmc`, retrieve and normalize reaction rates, and other perform other tasks. 

Note that instead of explicitly passing the path of the chain file to `CoupledOperator`, it can also be specified through `openmc.config`. We'll use this approach so that we don't need to specify it each time we're creating an operator:

We will then simulate our fuel pin operating at linear power of 174 W/cm, or 174 W given a unit height for our problem. For this problem, we will take depletion step sizes of 30 days, and instruct OpenMC to re-run a transport simulation every 30 days until we have modeled the problem over a six month cycle. Note that these values are not cumulative.

And lastly, we will use the basic predictor, or forward Euler, time integration scheme. Other, more advanced methods are provided to the user through `openmc.deplete`

To perform the simulation, we use the `integrate` method, and let `openmc` take care of the rest.

In [ ]:
integrator.integrate()

## Analyzing Results

The depletion simulation produces a few output files. First, the statepoint files from each individual transport simulation are written to `openmc_simulation_n<N>.h5`, where `<N>` indicates the current depletion step. Any tallies that we defined in `tallies.xml` will be included in these files across our simulations. We have 7 such files, one for each our of 6 depletion steps and the initial state.

In [ ]:
!ls openmc_simulation_n*.h5

The `depletion_results.h5` file contains information that is aggregated over all time steps through depletion. This includes the multiplication factor, as well as concentrations. We can process this file using the `openmc.deplete.Results` object

The first column of `k` is the value of `k-combined` at each point in our simulation, while the second column contains the associated uncertainty. We can plot this using `matplotlib`

In [ ]:
plt.errorbar(time, k[:, 0], yerr=k[:, 1])
plt.xlabel("Time (days)")
plt.ylabel("$Multplication Factor");

Due to the low number of particles selected, the uncertainty on each value is rather high. However, we can still see the decline over time due to fuel consumption.

We can then examine concentrations of atoms in each of our materials. 

In [ ]:
plt.plot(time, u5, label="U235")
plt.xlabel("Time [d]")
plt.ylabel("U235 concentration [atom/b-cm]");

In [ ]:
plt.plot(time, xe135, label="Xe135")
plt.xlabel("Time [d]")
plt.ylabel("Xe135 concentration [atom/b-cm]");

We can also examine reaction rates over time using the `Results`

In [ ]:
plt.plot(time, u5_fission)
plt.xlabel("Time [d]")
plt.ylabel("Fission reactions / s");

## Depletion Integrators

Using higher-order integrators, like the `CECMIntegrator`, `EPCRK4Integrator` with a fourth order Runge-Kutta, or the `LEQIIntegrator`, can improve the accuracy of a simulation, or at least allow you to take longer depletion steps between transport simulations with similar accuracy.

In [ ]:
long_steps = [60, 60, 60]

In [ ]:
results = openmc.deplete.Results('depletion_results.h5')
time, k = results.get_keff()
time /= (24 * 60 * 60)
plt.errorbar(time, k[:, 0], k[:, 1])

## Integration timestep/power options

In [ ]:


operator = openmc.deplete.CoupledOperator(model)
dep_int = openmc.deplete.CECMIntegrator(operator, timesteps, power, timestep_units='d')
dep_int.integrate()

In [ ]:
results = openmc.deplete.Results("depletion_results.h5")
time, i135 = results.get_atoms(fuel_material, "I135", time_units='h', nuc_units='atom/b-cm')
plt.errorbar(time[-4:], i135[-4:])

## Normalization (energy) modes

By default, the depletion chain has a set of fixed Q values that are used to estimate energy deposited from fission. However, this doesn't account for energy redistribution/loss from coupled neutron-photon transport or any incident neutron energy-dependent effects. Thus, you should also be aware that OpenMC gives you knobs to turn for how reaction rates are normalized based on observed energy deposition:

```Python
openmc.deplete.CoupledOperator(..., normalization_mode="fission-q")  # fixed Q values for fission
openmc.deplete.CoupledOperator(..., normalization_mode="energy-deposition")  # explicit heating tallies
openmc.deplete.CoupledOperator(..., normalization_mode="source-rate")  # for fixed source (activation) calculations
```

## Choice of depletion step size

A general rule of thumb is to use depletion step sizes around 2 MWd/kgHM, where kgHM is really the initial heavy metal mass in kg. If your problem includes integral burnable absorbers, these typically require shorter time steps at or below 1 MWd/kgHM. These are typically valid for the predictor scheme, as the point of recent schemes is to extend this step size. A good convergence study, where the step size is decreased until some convergence metric is satisfied, is a beneficial exercise.

We can use the `CoupledOperator` to determine our maximum step size using this recommendation. The `heavy_metal` attribute returns the mass of initial heavy metal in g, which, using our power, can be used to compute this step size. $$\frac{2\,MWd}{kgHM} = \frac{P\times\Delta}{hm_{op}}$$

In [ ]:
print(f"'Maximum' depletion step: {max_step:5.3} [d]")

## Differentiating burnable materials

OpenMC allows you to differentiate materials that reappear in multiple places. If we had built an entire core with the single `fuel` material, every pin would be depleted using the same averaged spectrum and reaction rates, which is physically incorrect. The `CoupledOperator` can differentiate these materials using the `diff_burnable_mats` argument. In the case, the volume assigned to the original material will be divided over each new instance of the material.

In [ ]:
new_model = openmc.Model()
new_model.settings = model.settings

# Create materials for fuel, clad, and water
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)
fuel.volume = 4 * math.pi * radii[0] ** 2

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
model.materials = openmc.Materials([fuel, clad, water])

# Create a fuel pin universe
radii = [0.42, 0.45]
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
materials = [fuel, clad, water]
pin_univ = openmc.model.pin(pin_surfaces, model.materials)

lattice = openmc.RectLattice()
lattice.lower_left = (-pitch, -pitch)
lattice.pitch = (pitch, pitch)
lattice.universes = [
    [pin_univ, pin_univ],
    [pin_univ, pin_univ]
]
lattice.outer = pin_univ

# Put it into a single cell defined as a box with reflective boundary conditions
bound_box = openmc.rectangular_prism(2*pitch, 2*pitch, boundary_type="reflective")
root_cell = openmc.Cell(fill=lattice, region=bound_box)
new_model.geometry = openmc.Geometry([root_cell])

new_model.geometry.root_universe.plot(width=(2*pitch, 2*pitch))

In [ ]:
new_op = openmc.deplete.CoupledOperator(new_model, diff_burnable_mats=True)
integrator = openmc.deplete.PredictorIntegrator(new_op, time_steps, power, timestep_units='d')
integrator.integrate()